## Homework 6: ELMO

#### Student name: Olha Tanyuk


In [1]:
#requires Tensorflow 2.0 and Tf-hub 0.7!!
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [2]:
print(tf.__version__)
print(hub.__version__)


2.0.0
0.7.0


Load up the ELMO model

In [3]:
module_url = "https://tfhub.dev/google/elmo/3"
embed = hub.load(module_url)

W1115 12:57:56.722823  2808 deprecation.py:506] From C:\Users\otany\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Example of how to get data from the model

In [4]:
text=tf.convert_to_tensor(["I am feeling kind of blue", "Blue is the color of the sky","lol wut"])
out = embed.signatures['default'](text)['elmo']



In [5]:
out.shape

TensorShape([3, 7, 1024])

In [6]:
out[0][5]

<tf.Tensor: id=6281, shape=(1024,), dtype=float32, numpy=
array([-0.5775769 , -0.7272091 ,  0.05588115, ...,  0.2270948 ,
        0.30447572,  0.3096969 ], dtype=float32)>

## Outputs

### The output dictionary contains:

  * word_emb: the character-based word representations with shape [batch_size, max_length, 512].
  * lstm_outputs1: the first LSTM hidden state with shape [batch_size, max_length, 1024].
  * lstm_outputs2: the second LSTM hidden state with shape [batch_size, max_length, 1024].
  * elmo: the weighted sum of the 3 layers, where the weights are trainable. This tensor has shape [batch_size, max_length, 1024]
  * default: a fixed mean-pooling of all contextualized word representations with shape [batch_size, 1024].


#### Q1:  Run a 4 words with different context meanings through Elmo--what is the (cosine) similarity of that word in the different contexts.

In [65]:
text1=tf.convert_to_tensor(["What was your net gain for the year", "Crabbing is easier if you bring a net along"])
out1 = embed.signatures['default'](text1)['elmo']
out1_1 = out1[0][3]
out1_2 = out1[1][7]

from scipy.spatial import distance
print ("the (cosine) similarity for word 'net' with two diff meanings:",(distance.cosine(out1_1, out1_2))*(-1)+1)

the (cosine) similarity for word 'net' with two diff meanings: 0.6077427268028259


In [66]:
text2=tf.convert_to_tensor(["They got engaged on March 7th", "The students were very engaged in the presentation"])
out2 = embed.signatures['default'](text2)['elmo']
out2_1 = out2[0][2]
out2_2 = out2[1][4]

print ("the (cosine) similarity for word 'engaged' with two diff meanings:",(distance.cosine(out2_1, out2_2))*(-1)+1)

the (cosine) similarity for word 'engaged' with two diff meanings: 0.8104850053787231


In [67]:
text3=tf.convert_to_tensor(["Joe took Alexandria out on a date", "What is your date of birth"])
out3 = embed.signatures['default'](text3)['elmo']
out3_1 = out3[0][6]
out3_2 = out3[1][3]

print ("the (cosine) similarity for word 'date' with two diff meanings:",(distance.cosine(out3_1, out3_2))*(-1)+1)

the (cosine) similarity for word 'date' with two diff meanings: 0.6532547473907471


In [68]:
text4=tf.convert_to_tensor(["My favorite flower is a rose", "He quickly rose from his seat"])
out4 = embed.signatures['default'](text4)['elmo']
out4_1 = out4[0][5]
out4_2 = out4[1][2]

print ("the (cosine) similarity for word 'rose' with two diff meanings:",(distance.cosine(out4_1, out4_2))*(-1)+1)

the (cosine) similarity for word 'rose' with two diff meanings: 0.45836153626441956


#### Q2:  Where is the padding?  Where does it go in the sequence?

In [40]:
out3_3 = out3[1][6]
out3_3

<tf.Tensor: id=96822, shape=(1024,), dtype=float32, numpy=
array([-0.02840841, -0.04353216,  0.04130162, ...,  0.02583168,
       -0.01429836, -0.01650422], dtype=float32)>

In [44]:
out2_3 = out2[0][6]
out2_4 = out2[0][7]
out2_3

<tf.Tensor: id=96854, shape=(1024,), dtype=float32, numpy=
array([-0.02840841, -0.04353216,  0.04130163, ...,  0.02583168,
       -0.01429836, -0.01650423], dtype=float32)>

In [45]:
out2_4

<tf.Tensor: id=96862, shape=(1024,), dtype=float32, numpy=
array([-0.02840841, -0.04353216,  0.04130163, ...,  0.02583168,
       -0.01429836, -0.01650423], dtype=float32)>

Looks like padding goes at the end of the sequence to match the longest sentence. Interesting, that the values in the padding arrays are the same for each padding for different sentences.

#### Q3 Describe how you would use an ELMO vector in a Neural Network below. Where does ELMO go? What else needs to be changed (added/removed)?

In [ ]:
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import  Lambda, LSTM as LSTM, SimpleRNN
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
numpy.random.seed(7)
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# embedding_vecor_length is not 300 any more, we will use 1024 in ElmoEmbedding
batch_size=128
model2 = Sequential()

# Embeding should be ELMO
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_review_length])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]
input_text = Input(shape=(max_review_length,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(None, max_review_length, 1024))(input_text)

# Change to Bidirectional LSTM
model2.add(Bidirectional(LSTM(512, return_sequences=True)))(embedding)

model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=128)

#### Q4 Compare the size of elmo vectors (dot product) with the size of spacy vectors (dot product)

In [51]:
text=tf.convert_to_tensor(["I am feeling kind of blue", "Blue is the color of the sky","lol wut"])
out = embed.signatures['default'](text)['elmo']

In [58]:
# dot product for words "feeling" and "kind" from one sentence using elmo
np.dot(out[0][2],out[0][3])

94.40007

In [53]:
# dot product for words "feeling" and "kind" using spacy
import spacy

nlp = spacy.load("en_core_web_md")
token1 = nlp("feeling")
token2 = nlp("kind")

np.dot(token1.vector,token2.vector)

20.884392

The size of spacy vectors (dot product) is smaller than the size of elmo vectors (dot product). Probably one of the reasons is that Elmo vector length is 1024 while the spacy vector length is 300. Lets check another pair of words.

In [59]:
# dot product for words "blue" and "color" from different sentences using elmo
np.dot(out[0][5],out[1][3])

66.9234

In [60]:
# dot product for words "blue" and "color" using spacy
token3 = nlp("blue")
token4 = nlp("color")

np.dot(token3.vector,token4.vector)

29.746302

We got the same result: the size of spacy vectors (dot product) is smaller than the size of elmo vectors (dot product).